Ideas for what to do next: 
- ~do Gitty stuff like repos and cloning and pushing~

- ~Try to get this working in Jupyter~

- ~Get the policies~

- Get all policy details related to climate?

- Loop through all division details to try to get & store them all

- Try to get just the divisions associated with the climate policies

- Try some python dataviz,  

- Get PowerBI licence and do some dataviz

- Get tweets and link them to the MPs

- Try to get data into redshift or mongodb

- Should I be storing the api key as a variable? I don't think so, should be fine to just have written out each time right?

# First some setup

In [1]:
#Import libraries
import requests
import json
import pandas as pd
import os

#pd.show_versions()
#pd.__version__

In [2]:
os.getcwd()

'C:\\Users\\Oona\\Data Projects\\climate\\Scripts'

# Now look at people

In [3]:
##get the list of people in Parliament into a dataframe

#Api call
mps_get = requests.get("https://theyvoteforyou.org.au/api/v1/people.json?key=HsjbG%2FoZdGMFMbN4PK%2F1")

#Print the api status to confirm the connection etc
print("api status: ", mps_get.status_code)

#pull the content from the api
mps_content = mps_get.content

#Load the content into a Python object
mps_loaded = json.loads(mps_content)

#normalize it (to flatten out the hierarchy) and create a pandas dataframe to Do Stuff With
mps = pd.json_normalize(mps_loaded)

mps.head()



api status:  200


,id,latest_member.id,latest_member.name.first,latest_member.name.last,latest_member.electorate,latest_member.house,latest_member.party
0,10007,6,Anthony,Albanese,Grayndler,representatives,Australian Labor Party
1,10014,13,Kevin,Andrews,Menzies,representatives,Liberal Party
2,10043,40,Sharon,Bird,Cunningham,representatives,Australian Labor Party
3,10081,69,Tony,Burke,Watson,representatives,Australian Labor Party
4,10088,74,Anthony,Byrne,Holt,representatives,Australian Labor Party


In [4]:
#This is writing & calling a function to view the JSON directly (to help clarify what the structure is prior to normalizing it)
#def jprint(obj):
    # create a formatted string of the Python JSON object
#   text = json.dumps(obj, sort_keys=True, indent=4)
#   print(text)
    
#jprint(mps_get.json())



In [5]:
#Exploring the dataset


#rename cols for readability
mps = mps.rename(columns={'latest_member.id':'member_id'})
mps.columns = mps.columns.str.replace('latest_member.','')
mps = mps.rename(columns={'name.first':'first_name','name.last':'last_name'})

print(mps.columns)

print(mps.groupby(['house','party']).size())
print(mps['party'].value_counts())

Index(['id', 'member_id', 'first_name', 'last_name', 'electorate', 'house',
       'party'],
      dtype='object')
house            party                            
representatives  Australian Greens                     1
                 Australian Labor Party               68
                 CWM                                   1
                 Centre Alliance                       1
                 Independent                           4
                 Katter's Australian Party             1
                 Liberal National Party                5
                 Liberal Party                        54
                 National Party                       15
                 SPK                                   1
senate           Australian Greens                     9
                 Australian Labor Party               25
                 Centre Alliance                       1
                 Country Liberal Party                 1
                 DPRES              

c:\users\oona\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [6]:
#Export the MPs to CSV

mps.to_csv("..\Data\mps.csv", index=False)

# Now we will switch to looking at some policies.

In [7]:
#Api call
policies_get = requests.get("https://theyvoteforyou.org.au/api/v1/policies.json?key=HsjbG%2FoZdGMFMbN4PK%2F1")

#Print the api status to confirm the connection etc
print("api status: ", policies_get.status_code)

#pull the content from the api
policies_content = policies_get.content

#Load the content into a Python object
policies_loaded = json.loads(policies_content)

#normalize it (to flatten out the hierarchy) and create a pandas dataframe to Do Stuff With
policies = pd.json_normalize(policies_loaded)

policies

api status:  200


,id,name,description,provisional
0,1,same-sex marriage equality,the Federal Government should amend the Marria...,False
1,2,tobacco plain packaging,the federal government should introduce plain ...,False
2,3,a carbon price,the federal government should introduce a carb...,False
3,4,increasing scrutiny of asylum seeker management,there should be more independent access to det...,False
4,5,government administered paid parental leave,the federal government should administer the p...,False
...,...,...,...,...
253,254,Scrapping the Cashless Welfare Card,the Federal Government should end this disastr...,True
254,255,increasing protections for franchisees,the federal government should amend current la...,False
255,256,an independent inquiry into Attorney-General C...,the federal government should establish an ind...,False
256,257,a constitutionally enshrined First Nations Voi...,the Constitution should be amended to include ...,False


In [8]:
#Export the policies to CSV

policies.to_csv("..\Data\policies.csv", index=False)

Now I'm going to look at filtering just to climate change policies. Looking through all of the policies, we really can't automate this fully as there's no consistent linguistic flag. It's also super subjective, we'll have to set SOME kind of parameters for what we consider "climate change related". For instance, "Increasing Aboriginal land rights" isn't directly climate change but could obviously have a big impact. Likewise any number of mining or mineral-export topics. Like... is supporting the mining industry anti-climate change? Also biodiversity... for instance, fishing restrictions are meant to keep fish stock stable/sustainable, but I assume that's also very related to climate change since more restrictions > higher prices > less fish consumption > better for climate change? but not if it's being replaced by, say beef consumption? Arghhh being in politics must be hard! For now I'm just gonna go with "very obviously climate change policy related" and I'll consider all these other things later!!

In [9]:
#Filter policies to those that include climate-related keywords in their name or description

#make list of keywords
keywords = ['carbon','renewable','emission','fossil','climate','sea level', 'nuclear']

#join the list with pipes to check against the 
word_filter = "|".join(keywords)

climate_policies = policies[policies['name'].str.contains(word_filter)|policies['description'].str.contains(word_filter)]

In [10]:
#Export the climate policies to CSV

climate_policies.to_csv("..\Data\climate_policies.csv", index=False)

# Now getting the details of the climate policies 

In [13]:
#We will need to build a... wait for it... function!!

#Api call
details_get = requests.get("https://theyvoteforyou.org.au/api/v1/policies/3.json?key=HsjbG%2FoZdGMFMbN4PK%2F1")

#Print the api status to confirm the connection etc
print("api status: ", details_get.status_code)

#pull the content from the api
details_content = details_get.content

#Load the content into a Python object
details_loaded = json.loads(details_content)

#normalize it (to flatten out the hierarchy) and create a pandas dataframe to Do Stuff With
details = pd.json_normalize(details_loaded)



api status:  200


,id,name,description,provisional,policy_divisions,people_comparisons
0,3,a carbon price,the federal government should introduce a carb...,False,"[{'division': {'id': 837, 'house': 'representa...","[{'person': {'id': 10004, 'latest_member': {'i..."


In [63]:
#Now we'll unnest the "people comparisons" column to find out who voted for these policies, and how much they agreed with them

#First we need to "explode" the people_comparisons nested array to create a new row for each person
exploded = details.drop(columns=['policy_divisions']).explode('people_comparisons').reset_index(drop=True)

#Then we need to "normalize" the nested person details into their own columns
normal = pd.json_normalize(exploded['people_comparisons'])

#Then concatenate to reattach the person detail columns to the policies
people_policies = pd.concat([exploded, normal], axis=1).drop(columns=['people_comparisons'])

OK So for now we've got all the people details attached to ONE policy. But we need a function to go through and do this for all of the policies. Tomorrow's work!